# **Configuration**

# **Installs Microsoft Graph SDK**

In [ ]:
#Installs Microsoft Graph SDK, used for programmatic 
# access to Microsoft 365 and Fabric services. 
# The -q flag hides unnecessary install logs.

%pip install -q msgraph-sdk

print("Required Python packages installed successfully")

debugMode = True                   # Enables debug printouts and logs
keyVaultName = "FabricAutomationKeyVault"  # Azure Key Vault name to fetch credentials from

StatementMeta(, 614fcdca-c1ed-4fb5-931c-45390e55f68c, 8, Finished, Available, Finished)


[notice] A new release of pip is available: 24.0 -> 25.3
[notice] To update, run: python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.
✅ Required Python packages installed successfully



# **LIB**

In [ ]:
import json
import os
import msgraph
import requests

from msgraph import GraphServiceClient
from azure.identity import DefaultAzureCredential
from azure.keyvault.secrets import SecretClient
from azure.identity import ClientSecretCredential
import sempy.fabric as F
import pprint
from notebookutils import mssparkutils
from datetime import datetime
import pandas as pd
from zoneinfo import ZoneInfo
from requests.auth import HTTPBasicAuth

print("All modules imported successfully")


StatementMeta(, 614fcdca-c1ed-4fb5-931c-45390e55f68c, 10, Finished, Available, Finished)

✅ All modules imported successfully


# **Key Vault**

In [ ]:
#Credentials - KeyVault

client_id = mssparkutils.credentials.getSecret("URL", "<KeyName>") 
client_secret = mssparkutils.credentials.getSecret("URL", "<KeyName>")
tenant_id = mssparkutils.credentials.getSecret("URL", "<KeyName>")


print("All modules imported and initialized successfully")

StatementMeta(, 614fcdca-c1ed-4fb5-931c-45390e55f68c, 11, Finished, Available, Finished)

✅ All modules imported and initialized successfully


In [ ]:
# === CONFIGURATION PARAMETERS FOR FABRIC WORKSPACE CREATION ===
# Modify these values as needed to define your workspace setup.

config = {
    "workspaceName": "AutoProject",       # Name of the workspace to create
    "fabricCapacityName": "mcaps",   # Fabric capacity (compute cluster) to use
    "devOpsOrgName": "codereashu",           # Azure DevOps organization name
    "devOpsProjectName": "Automation",       # DevOps project that stores the repo
    "devOpsRepoName": "Auto_repo1",        # Repository name where code resides
    "devOpsBranchName": "main",       # Branch to connect workspace to
    "adminGroups": "Fabric_Automation",  # Azure AD groups with Admin access
    "devOpsPAT": "<PATToken>", # Personal Access Token for Azure DevOps
     "gitProviderType": "AzureDevOps", # Git provider type
     "directoryName": "GitAuto", # Directory name in the repo for workspace files
     "tenantId": "tenant_id", # Azure AD Tenant ID
     "servicePrincipalClientId": "client_id", # Service Principal Client ID
     "servicePrincipalSecret": "client_secret",# Service Principal Secret
     "devOpsRepoUrl": "<URL>" # DevOps Repo URL
}




StatementMeta(, 614fcdca-c1ed-4fb5-931c-45390e55f68c, 12, Finished, Available, Finished)

# **Graph API AUTH**

In [5]:
#Credential for graph api

credential = ClientSecretCredential(
        tenant_id=tenant_id,
        client_id=client_id,
        client_secret=client_secret
   )

graph_scopes = ['https://graph.microsoft.com/.default']

graph_client = GraphServiceClient(credentials=credential, scopes=graph_scopes)


StatementMeta(, 614fcdca-c1ed-4fb5-931c-45390e55f68c, 13, Finished, Available, Finished)

# **Fabric API Auth**

In [ ]:
class ServicePrincipalTokenProvider:
    def __init__(self, tenant_id, client_id, client_secret):
        self.credential = ClientSecretCredential(
            tenant_id=tenant_id,
            client_id=client_id,
            client_secret=client_secret
        )
        # The scope for the API
        self.scope = "https://api.fabric.microsoft.com/.default"

    def __call__(self):
        token = self.credential.get_token(self.scope)
        return token.token

StatementMeta(, 614fcdca-c1ed-4fb5-931c-45390e55f68c, 14, Finished, Available, Finished)

In [ ]:
# Credential for Fabric API
token_provider = ServicePrincipalTokenProvider(
        tenant_id=tenant_id,
        client_id=client_id,
        client_secret=client_secret
)

# Create the Fabric client
fabric_client = F.FabricRestClient(token_provider=token_provider) # Initialize Fabric client with token provider


StatementMeta(, 614fcdca-c1ed-4fb5-931c-45390e55f68c, 15, Finished, Available, Finished)

# **Capacity Details**

In [8]:
capacities_response = fabric_client.get("/v1/capacities")
capacities = capacities_response.json()
capacity = [ current_capacity for current_capacity in capacities["value"] if current_capacity["displayName"] == config["fabricCapacityName"]][0]
print(capacity)


StatementMeta(, 614fcdca-c1ed-4fb5-931c-45390e55f68c, 16, Finished, Available, Finished)

{'id': 'e97ce983-c225-41d2-ba8c-54f9914c7246', 'displayName': 'mcaps', 'sku': 'F2', 'region': 'Australia East', 'state': 'Active'}


# **workspace Creation**

In [ ]:
workspace_name = config["workspaceName"]

# First, check if the workspace already exists
print(f"Checking if workspace '{workspace_name}' already exists...")

list_response = fabric_client.get("/v1/workspaces")
existing_workspaces = list_response.json().get("value", [])

workspace_id = None

# Search for a workspace with the same displayName
for ws in existing_workspaces:
    if ws.get("displayName") == workspace_name:
        workspace_id = ws.get("id")
        print(f"Workspace '{workspace_name}' already exists.")
        print(f"Existing Workspace ID: {workspace_id}")
        break

# If not found, create a new one
if not workspace_id:
    print(f"Creating new workspace '{workspace_name}'...")

    workspace_definition = {
        "displayName": workspace_name,
        "capacityId": capacity["id"]
    }

    workspace_creation_response = fabric_client.post(
        "/v1/workspaces",
        json=workspace_definition
    )

    # Parse response
    workspace_data = workspace_creation_response.json()
    print("Full workspace creation response:")
    print(workspace_data)

    workspace_id = workspace_data.get("id")
    print(f"Workspace '{workspace_name}' created successfully!")
    print(f"New Workspace ID: {workspace_id}")

# Final confirmation
print(f"Using Workspace ID: {workspace_id}")


StatementMeta(, 614fcdca-c1ed-4fb5-931c-45390e55f68c, 38, Finished, Available, Finished)

🔍 Checking if workspace 'AutoProject' already exists...
ℹ️ Workspace 'AutoProject' already exists.
🆔 Existing Workspace ID: b10c02d7-fc19-4770-bff3-240681085a88
🏁 Using Workspace ID: b10c02d7-fc19-4770-bff3-240681085a88


In [ ]:
# === REUSABLE ROLE ASSIGNMENT FUNCTION ===
def assign_role_to_workspace(workspace_id, principal_id, principal_type, role_name):
    
    payload = {
        "principal": {
            "id": principal_id,      # Azure AD Object ID of SP or Group
            "type": principal_type   # "ServicePrincipal", "Group", or "User"
        },
        "role": role_name           # "Admin", "Contributor", "Viewer"
    }

    # Use FabricRestClient directly
    response = fabric_client.post(f"/v1/workspaces/{workspace_id}/roleAssignments", json=payload)

    # --- Handle Response ---
    if response.status_code in [200, 201]:
        print(f"Successfully assigned '{role_name}' role to {principal_type}: {principal_id}")
        print("Response:", response.json())
    elif response.status_code == 409:
        print(f"Principal {principal_id} already has the '{role_name}' role — skipping.")
    else:
        print(f"Failed to assign role ({response.status_code}): {response.text}")


assign_role_to_workspace(
    workspace_id=workspace_id,
    principal_id="790067eb-d5fd-4f42-a860-9af0ece9adca",
    principal_type="Group",
    role_name="Admin"
)


StatementMeta(, 614fcdca-c1ed-4fb5-931c-45390e55f68c, 39, Finished, Available, Finished)

FabricHTTPException: 409 Conflict for url: https://api.fabric.microsoft.com//v1/workspaces/b10c02d7-fc19-4770-bff3-240681085a88/roleAssignments
Error: {"requestId":"95a2198a-b455-438f-aec3-ad54b173b537","errorCode":"PrincipalAlreadyHasWorkspaceRolePermissions","moreDetails":[{"relatedResource":{"resourceId":"790067eb-d5fd-4f42-a860-9af0ece9adca","resourceType":"Principal"}}],"message":"The provided principal already has a role assigned in the workspace","relatedResource":{"resourceId":"b10c02d7-fc19-4770-bff3-240681085a88","resourceType":"Workspace"}}
Headers: {'Cache-Control': 'no-store, must-revalidate, no-cache', 'Pragma': 'no-cache', 'Transfer-Encoding': 'chunked', 'Content-Type': 'application/json; charset=utf-8', 'x-ms-public-api-error-code': 'PrincipalAlreadyHasWorkspaceRolePermissions', 'Strict-Transport-Security': 'max-age=31536000; includeSubDomains', 'X-Frame-Options': 'deny', 'X-Content-Type-Options': 'nosniff', 'RequestId': '95a2198a-b455-438f-aec3-ad54b173b537', 'Access-C

In [ ]:


def get_devops_repo_id(org, project, repo, pat_token):

    devOpsUrl = f"https://dev.azure.com/{org}/{project}/_apis/git/repositories/{repo}?api-version=7.0"

    # Authentication: username can be anything, PAT is used as password
    response = requests.get(devOpsUrl, auth=HTTPBasicAuth('', pat_token))

    # Debug: check raw text if JSON decoding fails
    if response.status_code != 200:
        print(f"Failed to fetch repo info ({response.status_code})")
        print("Response Text:", response.text)
        response.raise_for_status()

    repoInfo = response.json()
    return repoInfo["id"]


# === EXAMPLE USAGE ===
repositoryId = get_devops_repo_id(
    org=config["devOpsOrgName"],
    project=config["devOpsProjectName"],
    repo=config["devOpsRepoName"],
    pat_token=config["devOpsPAT"]   # store your PAT in config
)

print(f"Retrieved DevOps Repository ID: {repositoryId}")


StatementMeta(, 614fcdca-c1ed-4fb5-931c-45390e55f68c, 40, Finished, Available, Finished)

📁 Retrieved DevOps Repository ID: 7806c000-b4d9-4c05-890d-4d5fa4df2a4b


In [ ]:
 # Ensure json library is available if not imported earlier

print("Starting Azure DevOps connection process...")

# --- SETUP (Includes Previous Fixes) ---
# Construct the full Azure DevOps Git repository URL
devOpsUrl = f"https://dev.azure.com/{config['devOpsOrgName']}/{config['devOpsProjectName']}/_git/{config['devOpsRepoName']}"
connection_name = "GitConnection"
connection_id = None

# --- 1. CHECK FOR EXISTING CONNECTION ---
print(f"Checking for existing connection with URL: {devOpsUrl}")

try:
    # Get all connections the Service Principal has access to
    list_response = fabric_client.get("/v1/connections")
    
    if list_response.status_code == 200:
        connections = list_response.json().get("value", [])
        
        # Iterate and find a match based on the unique URL parameter
        for conn in connections:
            if conn.get("connectivityType") == "ShareableCloud" and conn.get("displayName") == connection_name:
                
                # Extract URL from connectionDetails.parameters
                details = conn.get("connectionDetails", {})
                parameters = details.get("parameters", [])
                
                # Assuming the URL is the first parameter
                found_url = parameters[0].get("value") if parameters and len(parameters) > 0 else None
                
                if found_url == devOpsUrl:
                    connection_id = conn.get("id")
                    print(f"Existing connection '{connection_name}' found.")
                    print(f"Reusing Connection ID: {connection_id}")
                    break
    
except Exception as e:
    print(f"Error while retrieving existing connections: {e}")

# --- 2. CREATE NEW CONNECTION IF NOT FOUND ---
if not connection_id:
    print(f"Connection not found. Creating a new connection: {connection_name}")
    
    # Define payload for creation (Includes Fix #1: using variables)
    payload = {
        "displayName": connection_name,
        "connectivityType": "ShareableCloud",
        "connectionDetails": {
            "creationMethod": "AzureDevOpsSourceControl.Contents",
            "type": "AzureDevOpsSourceControl",
            "parameters": [
                {
                    "dataType": "Text",
                    "name": "url",
                    "value": devOpsUrl  # <-- Uses calculated devOpsUrl
                }
            ]
        },
        "credentialDetails": {
            "credentials": {
                "credentialType": "ServicePrincipal",
                "tenantId": tenant_id,
                "servicePrincipalClientId": client_id,
                "servicePrincipalSecret": client_secret
            }
        }
    }

    # Create connection
    connection_response = fabric_client.post("/v1/connections", json=payload)

    if connection_response.status_code in [200, 201]:
        connection_data = connection_response.json()
        print(f"Azure DevOps connection '{connection_name}' created successfully!")
        
        connection_id = connection_data.get("id")
        if connection_id:
            print(f"New Connection ID: {connection_id}")
        else:
            print("Connection created, but 'id' not found in response.")
    else:
        print(f"Failed to create Fabric connection (HTTP {connection_response.status_code})")
        print("Response:", connection_response.text)

# --- 3. LINK CONNECTION TO WORKSPACE ---
if connection_id:
    # Attach connection to workspace
    print(f"Linking connection {connection_id} with workspace {workspace_id}...")
    
    # Ensure all required parameters are present for the git connection
    link_payload = {
    "gitProviderDetails": {
        "organizationName": config["devOpsOrgName"],
        "projectName": config["devOpsProjectName"],
        "gitProviderType": config["gitProviderType"],
        "repositoryName": config["devOpsRepoName"],
        "branchName": config["devOpsBranchName"],
        "directoryName": config["directoryName"]
    },
    "myGitCredentials": {
    "source": "ConfiguredConnection",
    "connectionId":connection_id
  }
}
    
    link_response = fabric_client.post(
        f"/v1/workspaces/{workspace_id}/git/connect",
        json=link_payload
    )
#(f"/v1/workspaces/{workspace_id}/git/connect", json=payload)
    if link_response.status_code in [200, 201]:
        print(f"Workspace linked successfully with Azure DevOps Git repository!")
    elif link_response.status_code == 409:
        print(f"Workspace is already linked (HTTP 409). Skipping link step.")
    else:
        print(f"Workspace link failed (HTTP {link_response.status_code})")
        print("Response:", link_response.text)

else:
    print("Cannot proceed with linking. Connection ID is missing.")

StatementMeta(, 614fcdca-c1ed-4fb5-931c-45390e55f68c, 56, Finished, Available, Finished)

🔄 Starting Azure DevOps connection process...
🔎 Checking for existing connection with URL: https://dev.azure.com/codereashu/Automation/_git/Auto_repo1
✨ Connection not found. Creating a new connection: GitConnection


FabricHTTPException: 409 Conflict for url: https://api.fabric.microsoft.com//v1/connections
Error: {"requestId":"86bba4d0-09bd-4e14-b059-054e365e3d33","errorCode":"DuplicateConnectionName","message":"The connection DisplayName input is already being used by another connection"}
Headers: {'Cache-Control': 'no-store, must-revalidate, no-cache', 'Pragma': 'no-cache', 'Transfer-Encoding': 'chunked', 'Content-Type': 'application/json; charset=utf-8', 'x-ms-public-api-error-code': 'DuplicateConnectionName', 'Strict-Transport-Security': 'max-age=31536000; includeSubDomains', 'X-Frame-Options': 'deny', 'X-Content-Type-Options': 'nosniff', 'RequestId': '86bba4d0-09bd-4e14-b059-054e365e3d33', 'Access-Control-Expose-Headers': 'RequestId', 'request-redirected': 'true', 'home-cluster-uri': 'https://wabi-west-us3-a-primary-redirect.analysis.windows.net/', 'Date': 'Sat, 08 Nov 2025 05:48:04 GMT'}

In [49]:
Git_response = fabric_client.get(
        f"/v1/workspaces/{workspace_id}/git/connection")

Git_data = Git_response.json()
workspace_head = Git_data["gitSyncDetails"]["head"]
print(workspace_head)
print("🔹Git Connected successfully")
print(Git_data)


StatementMeta(, 614fcdca-c1ed-4fb5-931c-45390e55f68c, 57, Finished, Available, Finished)

de6b3cf6fbe1d2cb719e4333a094cdfde9652f3d
🔹Git Connected successfully
{'gitProviderDetails': {'organizationName': 'codereashu', 'projectName': 'Automation', 'gitProviderType': 'AzureDevOps', 'repositoryName': 'Auto_repo1', 'branchName': 'main', 'directoryName': '/GitAuto'}, 'gitSyncDetails': {'head': 'de6b3cf6fbe1d2cb719e4333a094cdfde9652f3d', 'lastSyncTime': None}, 'gitConnectionState': 'ConnectedAndInitialized'}


In [50]:
#POST https://api.fabric.microsoft.com/v1/workspaces/{workspaceId}/git/initializeConnection

Git_response = fabric_client.post(
        f"/v1/workspaces/{workspace_id}/git/initializeConnection")

Git_data = Git_response.json()
print("🔹Git Initialize successfully")
print(Git_data)


StatementMeta(, 614fcdca-c1ed-4fb5-931c-45390e55f68c, 58, Finished, Available, Finished)

FabricHTTPException: 409 Conflict for url: https://api.fabric.microsoft.com//v1/workspaces/b10c02d7-fc19-4770-bff3-240681085a88/git/initializeConnection
Error: {"requestId":"052c07b1-c0cf-4d65-9753-dab36947d062","errorCode":"WorkspaceGitConnectionAlreadyInitialized","message":"Connection between this workspace and Git has already been initialized.","relatedResource":{"resourceId":"b10c02d7-fc19-4770-bff3-240681085a88","resourceType":"Workspace"}}
Headers: {'Cache-Control': 'no-store, must-revalidate, no-cache', 'Pragma': 'no-cache', 'Transfer-Encoding': 'chunked', 'Content-Type': 'application/json; charset=utf-8', 'x-ms-public-api-error-code': 'WorkspaceGitConnectionAlreadyInitialized', 'Strict-Transport-Security': 'max-age=31536000; includeSubDomains', 'X-Frame-Options': 'deny', 'X-Content-Type-Options': 'nosniff', 'RequestId': '052c07b1-c0cf-4d65-9753-dab36947d062', 'Access-Control-Expose-Headers': 'RequestId', 'request-redirected': 'true', 'home-cluster-uri': 'https://wabi-west-us3-

In [ ]:
# === GET CURRENT GIT STATUS ===
# This call fetches all pending changes (like your new lakehouse)
# and gives you the new 'workspaceHead' required for the commit.

print("Fetching current Git status and pending changes...")

try:
    # Call the GET /git/status endpoint
    status_response = fabric_client.get(
        f"/v1/workspaces/{workspace_id}/git/status"
    )
    
    status_data = status_response.json()
    
    # Extract the new workspaceHead from the response
    workspace_head = status_data.get('workspaceHead')
    
    print("Successfully fetched status.")
    print(f"   New Workspace Head: {workspace_head}")
    
    # You can also see the changes it detected
    changes = status_data.get('changes', [])
    print(f"   Pending Changes Found: {len(changes)}")
    for change in changes:
        print(f"     - {change['itemMetadata']['displayName']} ({change['itemMetadata']['itemType']})")

except Exception as e:
    print(f"Failed to get Git status: {e}")
    print("   Run the 'initializeConnection' cell (Cell 24) just ONCE if this is the very first sync.")

StatementMeta(, 614fcdca-c1ed-4fb5-931c-45390e55f68c, 60, Finished, Available, Finished)

🔄 Fetching current Git status and pending changes...
✅ Successfully fetched status.
   New Workspace Head: de6b3cf6fbe1d2cb719e4333a094cdfde9652f3d
   Pending Changes Found: 1
     - commit (Lakehouse)


In [53]:
#POST https://api.fabric.microsoft.com/v1/workspaces/{workspaceId}/git/commitToGit

commit_payload = {
  "mode": "All",
  "workspaceHead": workspace_head,
  "comment": "I'm committing all my changes."
}

Git_commit = fabric_client.post(
        f"/v1/workspaces/{workspace_id}/git/commitToGit", json = commit_payload)

Git_Commit_data = Git_commit.json()
print("🔹Git commit successfully")
print(Git_Commit_data)

StatementMeta(, 614fcdca-c1ed-4fb5-931c-45390e55f68c, 61, Finished, Available, Finished)

🔹Git commit successfully
None
